# Gao's NDRS

1. Setup($1^k$)
2. KeyGen($\mathbb{P}$)
3. Sign($\mathbb{P}, R, {sk}_j, \mu$)
4. Verify($\mathbb{P}, R, \mu, \sigma$)
5. EvidenceGen($\mathbb{P}, R, {sk}_i, \mu, \sigma$)
6. EvidenceCheck($\mathbb{P}, R, i, \xi_i, \mu, \sigma$)

## Import

In [1]:
from ndrs import *

## NDRS

In [2]:
def original():
    # Define constants
    k = 10 # Sercurity parameter
    N = 5 # Number of signers

    # Create a new NDRS crypto object
    ndrs = NDRS(k)
    print(ndrs)

    keys = list[KeyPair]()
    for i in range(N):
        # Add a new signer
        keys.append(ndrs.key_gen())
        print(f"The key of Signer {i + 1} added: {keys[i]}")

    signer = keys[0]
    other_signers = [keys[2].pk, keys[3].pk]

    # Sign a message
    sig = ndrs.sign(signer, other_signers, "我最愛密碼學了！")
    print(f"Signature: {sig}")
    
    # Verify the signature
    if ndrs.verify("我最愛密碼學了！", sig):
        print("Signature is valid (accept)")
    else:
        print("Signature is invalid (reject)")

    # Generate an evidence
    evi = ndrs.evidence_gen(signer, "我最愛密碼學了！", sig)
    print(f"Evidence: {evi}")

    # Verify the evidence
    try:
        if ndrs.evidence_check(evi):
            print("Evidence is valid (confirmation)")
        else:
            print("Evidence is invalid (disavowal)")
    except Exception as e:
        print(f"Evidence is invalid (failed)")

## Attack

In [3]:
def attack():
    # Define constants
    k = 512 # Sercurity parameter
    N = 5 # Number of signers

    # Create a new NDRS crypto object
    ndrs = Frameable_NDRS(k)
    print(ndrs)

    keys = list[KeyPair]()
    for i in range(N):
        # Add a new signer
        keys.append(ndrs.key_gen())

    signer, fake_skey = ndrs.fake_skey_gen(keys[0])
    other_signers = [keys[2].pk, keys[3].pk]

    # Sign a message
    sig = ndrs.frameably_sign(KeyPair(signer.pk, fake_skey), other_signers, "我最愛密碼學了！", 0)
    
    
    # Verify the signature
    if ndrs.verify("我最愛密碼學了！", sig):
        print("Signature is valid")
    else:
        print("Signature is invalid")

    # Generate an evidence
    evi = ndrs.evidence_gen(signer, "我最愛密碼學了！", sig)

    # Verify the evidence
    try:
        if ndrs.evidence_check(evi):
            print("Evidence is valid (confirmation)")
        else:
            print("Evidence is invalid (disavowal)")
    except Exception as e:
        print(f"Evidence is invalid (failed)")

## Security Proof

In [ ]:
original()

In [ ]:
attack()